In [10]:
from typing import Callable
from dataclasses import dataclass
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import root
from scipy import linalg as la

# Implicit Runge-Kutta method

In [34]:
def RungeKuttaImplicit(f: Callable[[NDArray], NDArray], x0: NDArray, dt: float) -> NDArray:
    N = x0.size

    b1: float = 0.5
    b2: float = 0.5

    a11 = 0.25
    a12 = 0.25 - np.sqrt(3) / 6.0
    a21 = 0.25 + np.sqrt(3) / 6.0
    a22 = 0.25

    k1_res = lambda k1, k2: f(x0 + a11 * k1 * dt + a12 * k2 * dt) - k1
    k2_res = lambda k1, k2: f(x0 + a21 * k1 * dt + a22 * k2 * dt) - k2

    k1_0 = f(x0)
    k2_0 = f(x0)

    ks_0 = np.zeros(2 * N)
    ks_0[0:N] = k1_0
    ks_0[N:] = k2_0

    def residual(ks: NDArray) -> NDArray:
        res_vals = ks.copy()
        res_vals[0:N] = k1_res(ks[:N], ks[N:])
        res_vals[N:] = k2_res(ks[:N], ks[N:])

        return res_vals
    
    ks = root(residual, ks_0).x

    k1 = ks[0:N]
    k2 = ks[N:]

    return x0 + dt * (b1 * k1 + b2 * k2)

# Define Chemical System

In [35]:
@dataclass
class ChemicalState:
    conc: NDArray
    tot_conc: NDArray

@dataclass
class TotalConstants:
    tot_mat: NDArray

@dataclass
class KineticConstants:
    kin_mat: NDArray
    kin_rate: NDArray
    eq_const: NDArray

@dataclass
class EquilibriumConstants:
    eq_consts: NDArray
    stoich_mat: NDArray

# Solving Equilibrium

In [60]:
def SolveEquilibrium(chem: ChemicalState, eq: EquilibriumConstants, tot: TotalConstants) -> ChemicalState:
    N: int = chem.tot_conc.size
    x0: NDArray = np.zeros(N)

    log_x_p: NDArray = la.pinv(eq.stoich_mat) @ eq.eq_consts
    stoich_null: NDArray = la.null_space(eq.stoich_mat)

    def log_c(log_x: NDArray) -> NDArray:
        return log_x_p + stoich_null @ log_x
    
    def mass_err(log_x: NDArray) -> NDArray:
        c: NDArray = np.exp(log_c(log_x))
        return tot.tot_mat @ c - chem.tot_conc

    concVec = np.exp(log_c(root(mass_err, x0).x))
    
    return ChemicalState(concVec, chem.tot_conc)

# Kinetic Rates

In [61]:
def kinetic_rate(tot_conc: NDArray, surface_area: NDArray, eq: EquilibriumConstants, kin: KineticConstants, tot: TotalConstants) -> NDArray:
    c0 = np.zeros(eq.stoich_mat.shape[1])
    chem = ChemicalState(c0, tot_conc)
    chem_new = SolveEquilibrium(chem, eq, tot)
    conc: NDArray = chem_new.conc

    log_c = np.log(conc)
    log_iap = kin.kin_mat @ log_c
    iap = np.exp(log_iap)
    min_rates = surface_area * (kin.kin_rate ** 10.0) * (1.0 - iap / (kin.eq_const ** 10.0))
    min_rate_mat = np.diag(min_rates)
    component_mat = min_rate_mat @ kin.kin_mat
    spec_rate = component_mat.T @ np.ones(kin.kin_mat.shape[0])

    return tot.tot_mat @ spec_rate

def solve_kinetic(chem: ChemicalState, surface_area: NDArray, eq: EquilibriumConstants, kin: KineticConstants, tot: TotalConstants, dt: float) -> ChemicalState:
    def get_kin_rate(tot_conc: NDArray) -> NDArray:
        return kinetic_rate(tot_conc, surface_area, eq, kin, tot)
    
    new_tot_conc: NDArray = RungeKuttaImplicit(get_kin_rate, chem.tot_conc, dt)
    chem = ChemicalState(np.zeros(1), new_tot_conc)

    return SolveEquilibrium(chem, eq, tot)

In [62]:
dt: float = 500.0
kin_consts: NDArray = np.array([-9.19])
kin_eq_consts: NDArray = np.array([-7.3])
kin_mat: NDArray = np.array([[1, -1, 1, 0, 0]], dtype=float)
eq_consts: NDArray = np.array([9.617, -6.345])
stoich_mat: NDArray = np.array([
    [0, 0, 1, -1, -1],
    [-1, 1, 0, 0, 1]], dtype=float)

tot_mat: NDArray = np.array([
    [1, 2, 0, 0, 1],
    [0, 0, -1, 1, 0],
    [-2, 1, 0, 0, 1]], dtype=float)

conc_0: NDArray = np.array([0.001, 2.001e-3, 2.665e-10, 2e-3, 9.37e-16])
tot_0: NDArray = np.array([1e-3, 0.0, 0.002])
surf_area: NDArray = np.array([1.0])

eq = EquilibriumConstants(eq_consts=eq_consts, stoich_mat=stoich_mat)
kin = KineticConstants(kin_mat=kin_mat, kin_rate=kin_consts, eq_const=kin_eq_consts)
tot = TotalConstants(tot_mat=tot_mat)
chem = ChemicalState(conc_0, tot_0)

In [63]:
res = SolveEquilibrium(chem, eq, tot)

In [64]:
res

ChemicalState(conc=array([6.00336565e-213, 7.03544788e-213, 3.42507970e-005, 1.52097402e-006,
       1.49947370e-003]), tot_conc=array([0.001, 0.   , 0.002]))

In [45]:
solve_kinetic(chem, surf_area, eq, kin, tot, 500)

ChemicalState(conc=array([ 3.24422016,  0.30824049, 16.6660714 ,  0.06000214,  0.01849509]), tot_conc=array([-2.14844847e+12, -2.14844847e+12, -6.44534540e+12]))

In [5]:
b = np.array([-2.3953807570659746, -0.40024062618965134, -3.120616533579331])
A = np.array([
    [0.29053386811667137, -17.116443645797119, 4.2467072519070115],
    [-0.21268555267822675, -38.022506859078931, 6.208457901779596],
    [-0.077848315260808931, 36.513576808872017, -1.1128951883421223]
]).T

In [6]:
np.linalg.solve(A,b)

array([-5.47752879,  3.42030322,  0.98298914])